In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [0]:
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [0]:
from google.colab import drive
drive.mount('drive/')

In [0]:
votes = pd.read_csv("drive/My Drive/sna9/train3_data.csv")
votest = pd.read_csv("drive/My Drive/sna9/test1_data.csv")

In [5]:
item_count = votes.itemId.value_counts()
item_count.describe()

count    941.000000
mean       5.653560
std        2.678333
min        2.000000
25%        3.000000
50%        5.000000
75%        8.000000
max       24.000000
Name: itemId, dtype: float64

In [0]:
item_count.sort_index().head()

5     3
10    3
13    3
30    5
32    6
Name: itemId, dtype: int64

In [15]:
#data = pd.read_csv("drive/My Drive/sna9/dataiou.csv", index_col=0)
data.head()

,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true,iou,Xc,Yc,Xc_true,Yc_true,item_count,x,y,X,Y,iou2
itemId,,,,,,,,,,,,,,,,,,,
5,151,914,511,1331,198,827,649,1340,0.525002,331.0,1122.5,423.5,1083.5,3,167.666667,916.333333,506.166667,1289.833333,0.474264
10,72,559,427,858,26,605,433,847,0.736209,249.5,708.5,229.5,726.0,3,79.833333,560.833333,421.666667,856.666667,0.707669
13,46,600,621,981,233,698,622,920,0.394233,333.5,790.5,427.5,809.0,3,57.833333,601.500000,608.833333,973.833333,0.400978
30,65,353,324,491,61,368,329,530,0.640055,194.5,422.0,195.0,449.0,5,78.200000,359.300000,296.600000,475.100000,0.516166
32,9,279,388,884,30,536,328,813,0.360035,198.5,581.5,179.0,674.5,6,18.583333,417.500000,322.083333,820.416667,0.652876


In [0]:
data['item_count'] = item_count.sort_index()

Разные выжимки

In [0]:
data = data.reset_index()

In [9]:
data1 = votes.groupby("itemId")[['Xmin','Ymin','Xmax','Ymax', 'Xmin_true','Ymin_true','Xmax_true','Ymax_true']].mean()
data1.head()

,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true
itemId,,,,,,,,
5,184.333333,918.666667,501.333333,1248.666667,198.0,827.0,649.0,1340.0
10,87.666667,562.666667,416.333333,855.333333,26.0,605.0,433.0,847.0
13,69.666667,603.000000,596.666667,966.666667,233.0,698.0,622.0,920.0
30,91.400000,365.600000,269.200000,459.200000,61.0,368.0,329.0,530.0
32,28.166667,556.000000,256.166667,756.833333,30.0,536.0,328.0,813.0


In [0]:
data1['iou'] = data1.apply(intersection_over_union, axis=1)
data1['item_count'] = item_count.sort_index()

In [0]:
#data2 = votes.groupby("itemId")[['Xmin','Ymin','Xmax','Ymax', 'Xmin_true','Ymin_true','Xmax_true','Ymax_true']].median()
#data2['iou'] = data2.apply(intersection_over_union, axis=1)
data2['item_count'] = item_count.sort_index()
data2.head()

,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true,iou,item_count
itemId,,,,,,,,,,
5,192.0,917.0,510.0,1255.0,198.0,827.0,649.0,1340.0,0.451843,3
10,95.0,559.0,420.0,856.0,26.0,605.0,433.0,847.0,0.675867,3
13,69.0,603.0,608.0,969.0,233.0,698.0,622.0,920.0,0.415456,3
30,87.0,367.0,316.0,477.0,61.0,368.0,329.0,530.0,0.571910,5
32,30.0,560.0,254.0,759.5,30.0,536.0,328.0,813.0,0.541371,6


а макс.коробки в data, там уже есть iou и item_count

In [0]:
 # Сравним
data[data.item_count < 5.653560].iou.mean()  #  0.5687479503061718
#data[data.item_count > 5.653560].iou.mean() # 0.545680428704356

0.5687479503061718

при делении по средней частоте у редких точность выше

In [0]:
 # Сравним
#print(data[data.item_count < 4].iou.mean())  #  0.5737268721593892

for i in range(6,20,1):
    print(data[data.item_count > i].iou.mean())
    
# при > 15    0.6697669025380396

In [0]:
print(data[(data.item_count < 4) | (data.item_count > 15)].iou.mean()) 0.574527205745878

0.574527205745878


In [0]:
print(data[(data.item_count > 3) & (data.item_count < 16)].iou.mean()) 0.5465462298422379

0.5465462298422379


In [0]:
data.iou.mean() 0.5572509602623342

0.5572509602623342

лан... проверю мины

In [0]:
data1[data1.item_count < 4].iou.mean()  #  0.5571369542812175

0.5522674165667181

херня... а медианы

In [0]:
data2[data2.item_count > 6].iou.median()

0.47126436781609193

ладно. последняя вздорная идея - найти среднее между средними и максимуми

In [0]:
data = data.set_index('itemId')


In [0]:
data['x'] = (data.Xmin + data1.Xmin)/2
data['y'] = (data.Ymin + data1.Ymin)/2
data['X'] = (data.Xmax + data1.Xmax)/2
data['Y'] = (data.Ymax + data1.Ymax)/2

In [0]:
data['iou2'] = data[['x','y','X','Y', 'Xmin_true','Ymin_true','Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)


In [16]:
data['iou2'].mean()

0.560218362093997

In [0]:
data['iou'].mean()

0.5572509602623342

# TEST on TEST

In [21]:
#test1 = votest.groupby('itemId')[['Xmin','Ymin','Xmax','Ymax']].mean()
test1.head()

,Xmin,Ymin,Xmax,Ymax,x,y,X,Y
itemId,,,,,,,,
18,135.000000,704.000000,729.000000,1071.000000,130.500000,669.000000,746.500000,1101.500000
19,29.666667,560.666667,114.666667,725.666667,28.333333,559.833333,116.833333,727.333333
33,46.333333,393.666667,374.666667,633.666667,33.666667,386.833333,394.333333,642.833333
62,90.857143,964.857143,564.000000,1167.571429,62.928571,894.428571,601.500000,1214.785714
114,63.333333,575.000000,207.666667,767.000000,52.166667,572.500000,211.833333,773.500000


In [19]:
test2 = votest.groupby('itemId')[['Xmin','Ymin']].min()
test3 = votest.groupby('itemId')[['Xmax','Ymax']].max()
test2 = pd.concat([test2,test3], axis=1)
test2.head()

,Xmin,Ymin,Xmax,Ymax
itemId,,,,
18,126,634,764,1132
19,27,559,119,729
33,21,380,414,652
62,35,824,639,1262
114,41,570,216,780


In [0]:
test1['x'] = (test1.Xmin + test2.Xmin)/2
test1['y'] = (test1.Ymin + test2.Ymin)/2
test1['X'] = (test1.Xmax + test2.Xmax)/2
test1['Y'] = (test1.Ymax + test2.Ymax)/2

In [0]:
test1.to_csv("drive/My Drive/sna9/mean2max.csv", header=None)